<a href="https://colab.research.google.com/github/jmlly/curso-machine-learning/blob/main/Projeto/letterboxd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from google.colab import drive
import pandas as pd
import numpy as np

In [31]:
drive.mount('/content/drive')
dados_dos_filmes = pd.read_csv('/content/drive/MyDrive/machine-learning-project/movie_data.csv', lineterminator='\n')
usuario = pd.read_csv('/content/drive/MyDrive/machine-learning-project/users_export.csv')
avaliacao = pd.read_csv('/content/drive/MyDrive/machine-learning-project/ratings_export.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Visualização dos dados

### Tabela Usuário

In [32]:
usuario.head()

,_id,display_name,num_ratings_pages,num_reviews,username
0,5fc4172ec6cd28ebd99dd0e2,Lucy,32.0,1650,deathproof
1,5fc4172ec6cd28ebd99dd0ea,Matt Singer,52.0,1915,superpulse
2,5fc4172ec6cd28ebd99dd0ed,Sean Baker,21.0,1283,lilfilm
3,5fc4172ec6cd28ebd99dd0ee,iana,37.0,1177,ianamurray
4,5fc419171ebf67b9fbe48615,Lizzy,57.0,1810,punchdrunklizzy


In [33]:
usuario.columns

Index(['_id', 'display_name', 'num_ratings_pages', 'num_reviews', 'username'], dtype='object')

**_id** = ID exclusivo global.

**display_name** = Nome de exibição em Letterboxd no momento do último rastreamento.

**num_ratings_pages** = Número de páginas de avaliações.

**num_reviews** = Número total de avaliações feitas por este usuário.

**username** = Letterboxd ID/nome de usuário exclusivo. Pode ser usado para formar um URL Letterboxd.

In [34]:
usuario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8139 entries, 0 to 8138
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   _id                8139 non-null   object 
 1   display_name       7832 non-null   object 
 2   num_ratings_pages  4747 non-null   float64
 3   num_reviews        8139 non-null   int64  
 4   username           8139 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 318.1+ KB


In [35]:
usuario.isnull().sum()

_id                     0
display_name          307
num_ratings_pages    3392
num_reviews             0
username                0
dtype: int64

In [36]:
usuario.index.value_counts().sum()

8139

### Tabela Dados dos filmes

In [37]:
dados_dos_filmes.head()

,_id,genres,image_url,imdb_id,imdb_link,movie_id,movie_title,original_language,overview,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
0,5fc85f606758f69634496fd3,"[""Music"",""Animation""]",film-poster/4/6/4/4/4/0/464440-football-freaks...,NaN,NaN,football-freaks,Football Freaks,en,"Football crazy, football mad. Don’t watch this...",0.600,"[""United Kingdom""]",1971-12-05,0.0,[],535272.0,https://www.themoviedb.org/movie/535272/,0.0,0.0,1971.0
1,5fc85ff26758f696344ace0c,[],film-poster/2/4/5/5/0/0/245500-aftermath-0-230...,tt0586129,http://www.imdb.com/title/tt0586129/maindetails,aftermath-1960,Aftermath,en,Aftermath was the pilot for an unsold TV serie...,0.600,[],1960-04-17,22.0,[],318331.0,https://www.themoviedb.org/movie/318331/,8.0,1.0,1960.0
2,5fc85f606758f69634496fcd,"[""Drama""]",film-poster/9/3/3/1/8/93318-where-chimneys-are...,tt0045731,http://www.imdb.com/title/tt0045731/maindetails,where-chimneys-are-seen,Where Chimneys Are Seen,ja,Gosho’s most celebrated film both in Japan and...,1.568,"[""Japan""]",1953-03-05,108.0,"[""日本語""]",117779.0,https://www.themoviedb.org/movie/117779/,6.6,10.0,1953.0
3,5fc85f606758f69634496fd1,"[""Drama""]",NaN,tt0187327,http://www.imdb.com/title/tt0187327/maindetails,the-musicians-daughter,The Musician's Daughter,en,Carl Wagner's good wife was dying. His heart b...,0.600,"[""United States of America""]",1911-12-12,15.0,[],560377.0,https://www.themoviedb.org/movie/560377/,0.0,0.0,1911.0
4,5fc85f606758f69634496fd4,"[""Documentary""]",film-poster/4/5/4/6/0/3/454603-50-years-of-fab...,tt4769914,http://www.imdb.com/title/tt4769914/maindetails,50-years-of-fabulous,50 Years of Fabulous,en,50 Years of Fabulous recounts the rich history...,0.600,[],2018-05-17,75.0,[],525187.0,https://www.themoviedb.org/movie/525187/,0.0,0.0,2018.0


In [38]:
dados_dos_filmes.columns

Index(['_id', 'genres', 'image_url', 'imdb_id', 'imdb_link', 'movie_id',
       'movie_title', 'original_language', 'overview', 'popularity',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'tmdb_id', 'tmdb_link', 'vote_average', 'vote_count', 'year_released'],
      dtype='object')

**_id** = ID exclusivo global.

**genres** = Array  de gêneros de filmes associados.

**image_url** = Imagem usada no site Letterboxd para o filme. Geralmente um pôster.

**imdb_id** = ID para IMDB.

**imdb_link** = Link para a página IMDB do filme.

**movie_id** = Identificação do filme Letterboxd.

**movie_title** = Título de exibição.

**original_language** = Idioma original do filme

**overview** = Descrição do filme

**popularity** = Popularidade numérica 

**production_countries** = Matriz de países em que o filme foi produzido 

**release_date** = Data de lançamento

**runtime** = Tempo de execução em minutos

**spoken_languages** = Array de todos os idiomas falados no filme

**tmdb_id** = ID para acessar dados na API

**tmdb_link** = Link para a entrada do filme no TMDB

**vote_average** = Média de votos

**vote_count** = contagem de votos

**year_released** = Ano lançado

In [39]:
dados_dos_filmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285504 entries, 0 to 285503
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _id                   285504 non-null  object 
 1   genres                274872 non-null  object 
 2   image_url             264407 non-null  object 
 3   imdb_id               243802 non-null  object 
 4   imdb_link             243802 non-null  object 
 5   movie_id              285502 non-null  object 
 6   movie_title           283340 non-null  object 
 7   original_language     274872 non-null  object 
 8   overview              261248 non-null  object 
 9   popularity            274872 non-null  float64
 10  production_countries  274872 non-null  object 
 11  release_date          271050 non-null  object 
 12  runtime               270055 non-null  float64
 13  spoken_languages      274872 non-null  object 
 14  tmdb_id               279917 non-null  float64
 15  

In [40]:
dados_dos_filmes.index.value_counts().sum()

285504

### Tabela Avaliação

In [41]:
avaliacao.head()

,_id,movie_id,rating_val,user_id
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof
1,5fc57c5d6758f6963451a063,loving-2016,7,deathproof
2,5fc57c5d6758f6963451a0ef,scripted-content,7,deathproof
3,5fc57c5d6758f6963451a060,the-future,4,deathproof
4,5fc57c5c6758f69634519398,mank,5,deathproof


In [42]:
avaliacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11078167 entries, 0 to 11078166
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   _id         object
 1   movie_id    object
 2   rating_val  int64 
 3   user_id     object
dtypes: int64(1), object(3)
memory usage: 338.1+ MB


In [43]:
avaliacao.columns

Index(['_id', 'movie_id', 'rating_val', 'user_id'], dtype='object')

**_id** = ID exclusivo global.

**movie_id** = Letterboxd ID exclusivo para o filme. Pode ser usado para corresponder aos dados do filme neste conjunto de dados ou para construir um Letterboxd

**rating_val** = Valor da classificação com base nas estrelas dadas (em incrementos de meia estrela). Uma meia estrela é 1, cinco estrelas é 10, etc.

**user_id** = Letterboxd ID exclusivo para o usuário. Pode ser usado para corresponder aos dados do usuário neste conjunto de dados ou para construir um Letterboxd

In [44]:
avaliacao.index.value_counts().sum()

11078167

## Tratamento dos dados

plotar graficos sobre: número médio de reviews, generos, tempo medio dos filmes, pais de produção.